In [1]:
from src.helpers import DBConnector
import pandas as pd

In [3]:
db = DBConnector(
    "/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/ui/simulation/trusted.duckdb"
)

In [4]:
db.get_tables()

['mvc_crash', 'mvc_person', 'mvc_vehicles', 'nhtsa_safety_rating']

In this exploitation layer we would like "person_id", "person_age", "person_sex", "vehicle_make", "vehicle_year", "overall_rating". As we would like to be able to analyze which type of vehicles does people of certain age buy, and see if there exists any type of correlation between the vehicle and the probabilty of having and accident. 

In [5]:
mvc_crash = db.get_table_as_dataframe("mvc_crash")
mvc_person = db.get_table_as_dataframe("mvc_person")
mvc_vehicles = db.get_table_as_dataframe("mvc_vehicles")
nhtsa_safety_rating = db.get_table_as_dataframe("nhtsa_safety_rating")

In [6]:
mvc_vehicles.head()

,index,collision_id,vehicle_id,vehicle_type,vehicle_make,vehicle_model,vehicle_year,driver_sex,driver_license_status,driver_license_jurisdiction,...,overturned,right_front_bumper,right_front_quarter_panel,right_rear_bumper,right_rear_quarter_panel,right_side_doors,roof,trailer,undercarriage,crash_datetime
0,0,4691962,c767fb32-1d67-4806-beca-b2b4cb9bf422,station wagon,alfa romeo,NaN,2019.0,F,Licensed,PA,...,0,0,0,0,0,0,0,0,0,2024-01-01 02:30:00
1,1,4691806,e9465da7-880f-4af7-8406-b56aaf69b367,sedan,alfa romeo,NaN,2021.0,F,Licensed,NY,...,0,0,0,0,0,0,0,0,0,2024-01-01 20:00:00
2,11,4692049,5148fff9-d97f-4b32-b60b-773d0b6387f7,sedan,alfa romeo,NaN,2018.0,None,None,None,...,0,1,1,0,0,0,0,0,0,2024-01-01 05:10:00
3,16,4695404,bcde0ca8-9af4-4744-a51e-f5530cdbebda,2-door sedan,alfa romeo,NaN,NaN,None,None,None,...,0,0,0,0,0,0,0,0,0,2024-01-01 21:00:00
4,23,4691742,53f7ef2c-247d-4ee8-a9b7-49ddd00cc72a,station wagon,alfa romeo,NaN,2022.0,M,Licensed,NH,...,0,0,0,0,0,0,0,0,0,2024-01-01 11:39:00


In [8]:
vehicle_crash = mvc_vehicles.merge(
    mvc_crash,
    how="left",
    left_on="collision_id",
    right_on="collision_id",
)
vehicle_crash.head()

,index_x,collision_id,vehicle_id,vehicle_type,vehicle_make,vehicle_model,vehicle_year,driver_sex,driver_license_status,driver_license_jurisdiction,...,reaction_to_uninvolved_vehicle,steering_failure,tinted_windows,tire_failure/inadequate,traffic_control_disregarded,turning_improperly,unsafe_lane_changing,unsafe_speed,view_obstructed/limited,crash_datetime_y
0,0,4691962,c767fb32-1d67-4806-beca-b2b4cb9bf422,station wagon,alfa romeo,NaN,2019.0,F,Licensed,PA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01 02:30:00
1,1,4691806,e9465da7-880f-4af7-8406-b56aaf69b367,sedan,alfa romeo,NaN,2021.0,F,Licensed,NY,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01 20:00:00
2,11,4692049,5148fff9-d97f-4b32-b60b-773d0b6387f7,sedan,alfa romeo,NaN,2018.0,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01 05:10:00
3,16,4695404,bcde0ca8-9af4-4744-a51e-f5530cdbebda,2-door sedan,alfa romeo,NaN,NaN,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01 21:00:00
4,23,4691742,53f7ef2c-247d-4ee8-a9b7-49ddd00cc72a,station wagon,alfa romeo,NaN,2022.0,M,Licensed,NH,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2024-01-01 11:39:00


In [9]:
vehicle_crash = vehicle_crash[
    [
        "vehicle_make",
        "vehicle_year",
        "number_of_persons_injured",
        "number_of_persons_killed",
    ]
]

With the following step we will have all the number of persons injured by vehicle.

In [16]:
grouped_crashes = vehicle_crash.groupby(
    ["vehicle_make", "vehicle_year"], as_index=False
).sum()
grouped_crashes["num_crashes"] = vehicle_crash.groupby(
    ["vehicle_make", "vehicle_year"], as_index=False
).size()["size"]
grouped_crashes

,vehicle_make,vehicle_year,number_of_persons_injured,number_of_persons_killed,num_crashes
0,alfa romeo,1995.0,0.0,0.0,1
1,alfa romeo,1996.0,0.0,0.0,1
2,alfa romeo,1997.0,0.0,0.0,1
3,alfa romeo,1998.0,0.0,0.0,1
4,alfa romeo,2000.0,1.0,0.0,4
...,...,...,...,...,...
154,volvo,2013.0,0.0,0.0,1
155,volvo,2014.0,0.0,0.0,1
156,volvo,2019.0,0.0,0.0,1
157,volvo,2023.0,0.0,0.0,2


In [ ]:
grouped_crashes